# Automated Web Scraper For Amazon.com

## Installation

We will be using:

Selenium
BeautifulSoup
Webdrivers

In [1]:
# !pip install selenium
# !pip install msedge-selenium-tools
# !pip install bs4

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.22
    Uninstalling urllib3-1.22:
      Successfully uninstalled urllib3-1.22


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.


  Attempting uninstall: selenium
    Found existing installation: selenium 4.0.0
    Uninstalling selenium-4.0.0:
      Successfully uninstalled selenium-4.0.0


In [11]:
# pip install chromedriver-binary
!pip install chromedriver_binary==95.0.4638.69 

  Created wheel for chromedriver-binary: filename=chromedriver_binary-95.0.4638.69.0-py3-none-any.whl size=6029270 sha256=950e7d66fbd78524851ff210ca06fdd9545037db405c83e29d8fd80db51943ac
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\d3\df\b3\bf2d9576df30ad711a7393d71dde1ff62fa686432e24f3fede
Successfully built chromedriver-binary
  Attempting uninstall: chromedriver-binary
    Found existing installation: chromedriver-binary 96.0.4664.35.0
    Uninstalling chromedriver-binary-96.0.4664.35.0:
      Successfully uninstalled chromedriver-binary-96.0.4664.35.0


Import the necessary libraries

In [42]:
from selenium import webdriver
import chromedriver_binary

#for moicrosoft edge
from msedge.selenium_tools import Edge, EdgeOptions
import csv

Setup the web driver

In [43]:
#for chrome users
driver = webdriver.Chrome()

# driver = webdriver.Chrome(executable_path='/Macintosh HD/Users/mybeast/Dowmloads/chromedriver')

For Firefox and microsoft edge

In [14]:
## for Firefox

#driver = webdriver.Firefox()


##for microsoft edge

# options = EdgeOptions()
# options.use_chromium =True
# driver = Edge(options=options)

## reference:

## https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-win64.zip
## https://sites.google.com/a/chromium.org/chromedriver/

## https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/


Let's now specify the URL we will like to get data from. In this case: https://www.amazon.com/

In [15]:
url= 'https://www.amazon.com/'

In [16]:
driver.get(url)

We will define a function which we will later by just passing in our keyword to search on amazon.com as shown below

In [17]:
def my_url(keyword):
    
#     temp = 'https://www.amazon.com/s?k=phone+case&ref=nb_sb_noss_1' #let's get rid of the 'phone+case' and replace it with {} to make the url generic.
      temp = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1' # a template url
      keyword = keyword.replace(' ', '+')
      return temp.format(keyword)

In [18]:
#we can pass our keyword in the function to get the correct url of the keyword on amazon.com

url=my_url('laptop') 
url

'https://www.amazon.com/s?k=laptop&ref=nb_sb_noss_1'

In [19]:
#we can even pass multiple keywords
url=my_url('laptop charger')
url

'https://www.amazon.com/s?k=laptop+charger&ref=nb_sb_noss_1'

In [20]:
driver.get(url) #this will open in your browser and return the page for your keyword

In [21]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source,'html.parser')

In [22]:
soup_results=soup.find_all('div',{'data-component-type':'s-search-result'})

In [23]:
len(soup_results)

22

We will now create a template which we can based on to make a generic request to extract the data that we need.

In [24]:
# we will assign our first result to an obj

obj=soup_results[0]

In [25]:
atag = obj.h2.a #create the h2 tag variable

In [26]:
des = atag.text.strip()

In [27]:
des #we can see below that we have the title correctly scraped

'Mac Book Pro Charger - 96W USB C Power Adapter Compatible with MacBook Pro 16, 15, 13 Inch 2020 2019 2018, MacBook Air 13 Inch, Thunderbolt 3 Laptop Power Supply, Included 6.6ft USB-C to USB-C Cable'

In [28]:
#let's now create a generic url

url='https://www.amazon.com/'+atag.get('href')

In [29]:
# Get the Price
#let's get the price same way we searched for the title by looking for the div tag, in this case, we will look for the tag that contains the price of the item.

#we will get this from the 'span' which contains the a-price and then use the 'span' which contains 'a-offscreen' to obtain the actual price.

parent=obj.find('span','a-price')

price=parent.find('span','a-offscreen').text

price

'$35.99'

Get the Reviews

In [30]:
#We will do the same thing for the Reviews


rate=obj.find('span','a-icon-alt').text
rate

'4.7 out of 5 stars'

In [31]:
# alternatively
obj.i.text

'4.7 out of 5 stars'

Get the review Counts

In [34]:
#we can get the number of customers who have reviewed the item as well

counts_review=obj.find('span','a-size-base').text
counts_review

'380'

In [35]:
obj.img

<img alt="Sponsored Ad - Mac Book Pro Charger - 96W USB C Power Adapter Compatible with MacBook Pro 16, 15, 13 Inch 2020 2019 2018, ..." class="s-image" data-image-index="1" data-image-latency="s-product-image" data-image-load="" data-image-source-density="1" src="https://m.media-amazon.com/images/I/51iq26SKZsL._AC_UY218_.jpg" srcset="https://m.media-amazon.com/images/I/51iq26SKZsL._AC_UY218_.jpg 1x, https://m.media-amazon.com/images/I/51iq26SKZsL._AC_UY327_FMwebp_QL65_.jpg 1.5x, https://m.media-amazon.com/images/I/51iq26SKZsL._AC_UY436_FMwebp_QL65_.jpg 2x, https://m.media-amazon.com/images/I/51iq26SKZsL._AC_UY545_FMwebp_QL65_.jpg 2.5x, https://m.media-amazon.com/images/I/51iq26SKZsL._AC_UY654_FMwebp_QL65_.jpg 3x"/>

# Generic Fuction

We will now create a generic fuction for the sub codes to extract all the data at once.

In [44]:
from selenium import webdriver
import chromedriver_binary
from bs4 import BeautifulSoup
#for moicrosoft edge
from msedge.selenium_tools import Edge, EdgeOptions
import csv

#We will be using functions to achieve this

def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    keyword = keyword.replace(' ', '+')
    
    # Add Term Query To URL
    url = temp.format(keyword)
    
    # Add Page Query Placeholder
    url += '&page{}'
    
    return url

def extract_record(obj):
    atag = obj.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #it is possible that some items on amazom.com might not be having one of the items we are looking for(e.g. some items might not be having ratings or price), we will be getting error if we dont take care of that. We will therefore add some error handlers
    #if there are no price,probably the item is out of stock or not available, then we will ignore the item, but if there are no reviews yet, it's fine, we will still want to extract the item.
    try:
        parent=obj.find('span','a-price')
        price=parent.find('span','a-offscreen').text
    except AttributeError: #we are excepting the error if it occurs so that we can move to extract the next item, else the program will stop running and gives error
        return
    
    try:
        rate=obj.i.text
        counts_review = obj.find('span','a-size-base').text
    except AttributeError:
        #assigning empty string to ratings and 
        rate = ''
        counts_review = ''
    
    image = obj.find('img', {'class': 's-image'}).get('src') 
    
    #let's create a tuple that will contain all these items and assign it to a result variable
    result = (description, price, rate, counts_review, url,image)
    return result

'''Run Main Program Routine'''
def main(keyword):
    # Startup The Webdriver
    driver = webdriver.Chrome()
#     options = EdgeOptions()
#     options.use_chromium =True
#     driver = Edge(options=options)
    
    records = []  #an empty records list to contain all of our extracted records
    url =my_url(keyword)
    
    for page in range(1, 3):
        driver.get(url.format(page))
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        results=soup.find_all('div',{'data-component-type':'s-search-result'})
#         results=soup.find_all('div',{'data-component-type': 's-search-result'}) #same as we did above

        
#we will like to check if what we have return from the extract_record function is empty or not
        for item in results:
            record = extract_record(item) 
            if record: #if the record has something in it append to records list
                records.append(record) 
                
#         driver.quit()
    
#     # Save Results To CSV File
        with open('Results.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Description', 'Price', 'Rating', 'Reviews Count', 'URL','Image link'])
            writer.writerows(records)

In [45]:
main('school bag') #we can search for any item to extract data

You can scrape as many data points as possible with any keyword, oh yes !!

You can load the dataset you have scraped and perform and use it for your project

In [47]:
import pandas as pd
df=pd.read_csv('Results.csv')
df

,Description,Price,Rating,Reviews Count,URL,Image link
0,"Modoker Vintage Laptop Backpack for Women Men,...",$29.99,4.6 out of 5 stars,"17,837",https://www.amazon.com/gp/slredirect/picassoRe...,https://m.media-amazon.com/images/I/81ZhsL-772...
1,"Matein Travel Laptop Backpack, Business Anti T...",$29.99,4.8 out of 5 stars,"49,588",https://www.amazon.com/Backpack-Business-Charg...,https://m.media-amazon.com/images/I/81p91cnIFE...
2,"Laptop Backpack,Business Travel Anti Theft Sli...",$31.99,4.7 out of 5 stars,"15,918",https://www.amazon.com/Backpack-Business-Charg...,https://m.media-amazon.com/images/I/81idlqFqcU...
3,JanSport SuperBreak One Backpack - Lightweight...,$36.00,4.7 out of 5 stars,"23,810",https://www.amazon.com/JanSport-Superbreak-Bac...,https://m.media-amazon.com/images/I/91HkBT96VM...
4,Himawari Travel School Backpack with USB Charg...,$40.99,4.7 out of 5 stars,"12,254",https://www.amazon.com/Himawari-Travel-Backpac...,https://m.media-amazon.com/images/I/61RX9JQlk+...
5,"YOREPEK Backpack for Men,Extra Large 50L Trave...",$39.99,4.8 out of 5 stars,"25,419",https://www.amazon.com/Backpack-Charging-Frien...,https://m.media-amazon.com/images/I/71cxnnsEHl...
6,UGRACE Vintage Laptop Backpack with USB Chargi...,$29.99,4.7 out of 5 stars,"4,773",https://www.amazon.com/UGRACE-Backpack-Chargin...,https://m.media-amazon.com/images/I/81PH0selXq...
7,MANCIO Slim Laptop Backpack with USB Charging ...,$27.99,4.7 out of 5 stars,"7,399",https://www.amazon.com/Backpack-Charging-Resis...,https://m.media-amazon.com/images/I/7112BpcDsn...
8,PUMA Evercat Contender Backpack,$24.00,4.7 out of 5 stars,"9,592",https://www.amazon.com/PUMA-Evercat-Contender-...,https://m.media-amazon.com/images/I/A1Y1RCRyK2...
9,"Leather Messenger Bag for Men, 17.3 Inch Vinta...",$45.99,4.6 out of 5 stars,"2,466",https://www.amazon.com/gp/slredirect/picassoRe...,https://m.media-amazon.com/images/I/71eSTfEnl+...
